In [6]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pandas import json_normalize
import pickle

In [9]:
df1 = pd.read_csv('Top all.csv')
df2 = pd.read_csv('Song_Clusters.csv')

In [26]:
df1.head()

,Title,Artist
0,Margaritaville,Jimmy Buffett
1,Come Monday,Jimmy Buffett
2,Rich Men North of Richmond,Oliver Anthony Music
3,All Star,Smash Mouth
4,Cheeseburger In Paradise,Jimmy Buffett


In [30]:
df2.head()

,artists,track_name,clusters
0,Queen,Bohemian Rhapsody - Remastered 2011,1
1,Deep Purple,Child in Time,0
2,Meat Loaf,Paradise By the Dashboard Light,2
3,The Moody Blues,Nights In White Satin - Single Version / Mono,0
4,Aphrodite's Child,Rain And Tears,1


In [12]:
print(df1.shape)
print(df2.shape)

(600, 2)
(124643, 3)


In [4]:
song_name = input("Please enter the name of a song: ")

# Check if the input contains at least one alphabetical character and/or alphanumeric character
if any(c.isalpha() for c in song_name) and any(c.isalnum() for c in song_name):
    print(f"You entered a valid song name: {song_name}")
else:
    print("Invalid input. Please enter a song name containing letters and/or numbers.")

In [63]:
def recommender(song, df1, df2):
    
    # Load Spotify secrets
    secrets_file = open("secrets.txt", "r")
    string = secrets_file.read()
    secrets_dict = {}
    for line in string.split('\n'):
        if len(line) > 0:
            secrets_dict[line.split(':')[0]] = line.split(':')[1].strip()

    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                               client_secret=secrets_dict['clientsecret']))

    results = sp.search(q='track:' + song, type='track')
    track = pd.json_normalize(results["tracks"]["items"])
    if not track.empty:
        song_id = track.iloc[0]['id']
    else:
        print("Song not found in Spotify.")
        return
    audio_features_list = sp.audio_features(song_id)
    audio_features_df = pd.json_normalize(audio_features_list)
    
    audio_features_df = audio_features_df[['duration_ms', 'danceability',
                                           'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
                                           'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]

    # Load scaler and transform data
    with open('scaler.pkl', 'rb') as f:
        loaded_scaler = pickle.load(f)
        X = loaded_scaler.transform(audio_features_df)
    
    # Load KMeans model and predict cluster
    with open('kmeans.pkl', 'rb') as f:
        loaded_kmeans = pickle.load(f)
        clusters = loaded_kmeans.predict(X)
        
    matching_song = df1[(df1['Title'].str.lower() == song.lower())]

    if not matching_song.empty:
        print(f"The song '{matching_song['Title'].values[0]}' by {matching_song['Artist'].values[0]} matches your input.")

        recommended_song = df1[df1['Title'].str.lower() != song].sample(1)
        print(f"Here's a recommended song: '{recommended_song['Title'].values[0]}' by {recommended_song['Artist'].values[0]}")
    else:
        cluster_df = df2[df2['clusters'] == clusters[0]]
        recommended_song2 = cluster_df[cluster_df['track_name'].str.lower() != song].sample(1)
        print(f"Here's a recommended song: '{recommended_song2['track_name'].values[0]}' by {recommended_song2['artists'].values[0]}")

In [65]:
song_name = input("Please enter the name of a song: ")
recommender(song_name,df1,df2)

Please enter the name of a song: falling in love with you
Here's a recommended song: 'Sétimo Céu - Ao Vivo' by Geraldo Azevedo
